In [0]:
%sql
-- ETL Mapping: ADS_SMA_PROCESS_EVENTS_MAIN
-- Generated from DWHP logs (ads_lib_etl_owner.etl_task_logs)
-- Export date: 2025-09-11 12:12:26

/*LOAD_STAGE
INSERT INTO gap_catalog.ads_etl_owner.XC_SMA_PROCESS_EVENTS_MAIN 
            with ca0 as (
            select 
              s.eventid,
              unix_ts_to_date(TIMESTAMP/1000) dt_timestamp,
              unix_ts_to_date(TIMESTAMP/1000) t_timestamp,
              s.type type,
              s.cidla,
              cp.caseph_key,
              CP.case_key,
              s.USER as USER,
              HUMANTASKCLUID AS CLUID,
              TAKETYPE as taketype,
              null as eve_status_desc
            from gap_catalog.DS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            left join gap_catalog.ads_owner.case_phases cp on s.HUMANTASKID = cp.caseph_source_id
            and  cp.caseph_source_system_id = 'SMA'
            and  cp.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
            and cp.cpcat_key=2 and cp.CASEPH_VALID_TO=date'3000-01-01'
            where
            s.type like ('HT_%') or
            s.type in ('REQUISITION_CREATED','REQUISITION_COMPLETED')

            union  all

            select 
              s.eventid,
              unix_ts_to_date(TIMESTAMP/1000) dt_timestamp,
              unix_ts_to_date(TIMESTAMP/1000) t_timestamp,
              s.type type,
              s.cidla,
              a.caseph_key as caseph_key,
              nvl(a.case_key, -1) as case_key,
              s.USER as USER,
              coalesce(THECASECLUID, HUMANTASKCLUID, GEORGETASKCLUID) as cluid,
              TAKETYPE as taketype,
              null as eve_status_desc
            from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            left join gap_catalog.ads_owner.case_phases a 
            on a.caseph_source_id= s.SERVICEID
            and  a.caseph_source_system_id = 'SMA'
            and  a.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
            and a.cpcat_key=6 and a.CASEPH_VALID_TO=date'3000-01-01'
            --EVENT filter
            where s.type like ('ACTIVITY_%')

            union  all

            select
              s.eventid,
              unix_ts_to_date(TIMESTAMP/1000) dt_timestamp,
              unix_ts_to_date(TIMESTAMP/1000) t_timestamp,
              s.type type,
              s.cidla,
              CT.caseph_key as caseph_key,
              nvl(CT.case_key, -1) as case_key,
              s.USER as USER,
              coalesce(THECASECLUID, HUMANTASKCLUID, GEORGETASKCLUID) as cluid,
              TAKETYPE as taketype,
              null as eve_status_desc              
            from gap_catalog.ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            left join ads_owner.case_phases ct on CLIENTTASKID = ct.caseph_source_id
            and  ct.caseph_source_system_id = 'SMA'
            and  ct.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
            and ct.cpcat_key=7 and ct.CASEPH_VALID_TO=date'3000-01-01'
            --EVENT filter
            where s.type like ('CLIENT_TASK%')

            union  all

            select 
              s.eventid,
              unix_ts_to_date(TIMESTAMP/1000) dt_timestamp,
              unix_ts_to_date(TIMESTAMP/1000) t_timestamp,
              s.type type,
              s.cidla,
              Ca.caseph_key as caseph_key,
              nvl(CA.case_key, -1) as case_key,
              s.USER as USER,
             coalesce(THECASECLUID, HUMANTASKCLUID, GEORGETASKCLUID) as cluid,
              TAKETYPE as taketype,
              null as eve_status_desc
            from ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            left join ads_owner.case_phases ca on s.CLIENTACTIVITYID = ca.caseph_source_id
            and  ca.caseph_source_system_id = 'SMA'
            and  ca.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
            and ca.cpcat_key=8 and ca.CASEPH_VALID_TO=date'3000-01-01'
            --EVENT filter
            where s.type like ('CLIENT_ACTIVITY%')

            union  all

            select
              s.eventid,
              unix_ts_to_date(s.TIMESTAMP/1000) dt_timestamp,
              unix_ts_to_date(s.TIMESTAMP/1000) t_timestamp,
              s.type type,
              s.cidla,
              GT.caseph_key as caseph_key,
              nvl(GT.case_key, -1) as case_key,
              s.USER as USER,
             coalesce(s.THECASECLUID, s.HUMANTASKCLUID, s.GEORGETASKCLUID) as cluid,
              s.TAKETYPE as taketype,
              null as eve_status_desc
            from ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            left join ads_owner.case_phases gt on s.GEORGETASKID = gt.caseph_source_id
            and  gt.caseph_source_system_id = 'SMA'
            and  gt.caseph_source_sys_origin = 'SMA_MONITOR_EVENTS'
            and gt.cpcat_key=314 and gt.CASEPH_VALID_TO=date'3000-01-01' 
            --EVENT filter
            where s.type like ('GEORGE%')

            union all

            select 
                s.eventid,
                unix_ts_to_date(s.TIMESTAMP/1000) dt_timestamp,
                unix_ts_to_date(s.TIMESTAMP/1000) t_timestamp,
                s.type type,
                s.cidla,
                -1 as caseph_key,
                NULL case_key,
                s.USER as USER,
                coalesce(s.THECASECLUID, s.HUMANTASKCLUID, s.GEORGETASKCLUID) as cluid,
                s.TAKETYPE as taketype,
              s.label as eve_status_desc
            from ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            --EVENT filter
            where  
             S.TYPE LIKE ('RUNNER%')

           union all

            select 
                s.eventid,
                unix_ts_to_date(s.TIMESTAMP/1000) dt_timestamp,
                unix_ts_to_date(s.TIMESTAMP/1000) t_timestamp,
                s.type type,
                s.cidla,
                -1 as caseph_key,
                NULL case_key,
                s.USER as USER,
                coalesce(s.THECASECLUID, s.HUMANTASKCLUID, s.GEORGETASKCLUID) as cluid,
                s.TAKETYPE as taketype,
              null as eve_status_desc
            from ADS_ETL_OWNER.STG_SMA_PROCESS_EVENTS_PARSE s
            --EVENT filter
            where s.type like ('CASE%') 

            ) 
            select
              ca0.eventid as eve_source_id,
              'SMA' as eve_source_system_id,
              'SMA_MONITOR_EVENTS' as eve_source_sys_origin,
              nvl(P1.PT_KEY, -1) as pt_key,
              nvl(P1.PTORI_KEY, -1) as ptori_key,
              nvl(p1.PT_UNIFIED_KEY, -1) as pt_unified_key,
              nvl(p.PT_KEY,-1) as pt_employee_key,
              nvl(p.PTORI_KEY,-1) as ptori_employee_key,
              'XNA' as id_financial_product,
              ca0.caseph_key,
              -1 as ntp_key,
              -1 as chan_key,
              nvl(et.evetp_key, -1) as evetp_key,
              -1 as est_key,
              coalesce(c.case_key, ca0.case_key, -1) as case_key,
              trunc(ca0.dt_timestamp) as eve_date,
              ca0.t_timestamp as eve_datetime,
              ca0.t_timestamp as eve_recorded_date,
              'XNA' as proce_target,
              nvl(sr.str_key, -1) as str_key,
              nvl(a.CGP_ID, 'XNA') as cgp_id, nvl(a.ACC_KEY, -1) as acc_key, nvl(a.ACCTP_KEY, -1) as acctp_key,
              nvl(a.CARD_KEY, -1) as card_key, nvl(a.SRV_KEY, -1) as srv_key, nvl(c.CASE_MASTER_KEY, -1) as case_master_key, -1 as doc_key,
              -1 as docl_key ,
              eve_status_desc

            from ca0
            left join ads_owner.cases c on ca0.cidla = c.case_source_id  and c.case_source_system_id = 'SMA' and c.case_source_sys_origin = 'SMA_MONITOR_EVENTS' and c.case_valid_to=date'3000-01-01'
            left join dwh_owner.parties p on upper(ca0.USER)=p.pt_source_id and p.ptori_key = 32 and p.pt_source_system_id='LD0'
            left join dwh_owner.parties p1 on p1.PT_UNIQUE_PARTY_ID = ca0.cluid and p1.ptori_key = 4 and p1.pt_source_system_id='CR0'
            left join ads_owner.event_types et on et.evetp_source_id = ca0.type and et.evetp_source_system_id = 'SMA' and  et.evetp_source_sys_origin = 'SMA_SYSTEM_EVENTS' and et.evetp_valid_to=date'3000-01-01' 
            left join ads_owner.status_reason sr on ca0.taketype= sr.STR_SOURCE_ID and  sr.STR_SOURCE_SYSTEM_ID='SMA' and  sr.STR_SOURCE_SYS_ORIGIN='SMA_SYSTEM_EVENTS.TAKE_TYPE' and  sr.STR_VALID_TO =date'3000-01-01'
            left join (SELECT 
                PROD_INST_CASES.CASE_KEY AS CASE_KEY ,
                PROD_INST_CASES.ACC_KEY AS ACC_KEY ,
                PROD_INST_CASES.ACCTP_KEY AS ACCTP_KEY ,
                PROD_INST_CASES.CARD_KEY AS CARD_KEY ,
                PROD_INST_CASES.SRV_KEY AS SRV_KEY ,
                PROD_INST_CASES.CGP_ID AS CGP_ID ,
                rank() over (partition by PROD_INST_CASES.CASE_KEY order by PROD_INST_CASES.ACC_KEY desc, PROD_INST_CASES.CARD_KEY desc, case when PROD_INST_CASES.CGP_ID <> 'XNA' then 0 else 10 end, PROD_INST_CASES.PRICASE_VALID_FROM) AS rn   
              FROM 
                ADS_OWNER.PROD_INST_CASES PROD_INST_CASES  INNER JOIN  ADS_OWNER.PROD_INST_CASE_TYPES PROD_INST_CASE_TYPES  
                  ON  PROD_INST_CASES.PRICASETP_KEY = PROD_INST_CASE_TYPES.PRICASETP_KEY
              WHERE
                (to_date('20250910','yyyymmdd') between PROD_INST_CASES.PRICASE_VALID_from and PROD_INST_CASES.PRICASE_VALID_TO
              ) AND   (PROD_INST_CASE_TYPES.PRICASETP_VALID_TO = date '3000-01-01'
              and PROD_INST_CASE_TYPES.PRICASETP_SOURCE_SYSTEM_ID = 'ADS'
              and PROD_INST_CASE_TYPES.PRICASETP_SOURCE_ID = 'SMA_CASE_CGP')) a on c.case_key=a.case_key and a.rn=1            
;
*/
/*MERGE*/
merge  into gap_catalog.ADS_OWNER.PROCESS_EVENTS trg
using
(select /*+ full(xc) */  EVE_SOURCE_ID, 
   EVE_SOURCE_SYSTEM_ID, 
   EVE_SOURCE_SYS_ORIGIN, 
   PT_KEY, 
   PTORI_KEY, 
   PT_UNIFIED_KEY, 
   PT_EMPLOYEE_KEY, 
   PTORI_EMPLOYEE_KEY, 
   ID_FINANCIAL_PRODUCT, 
   CASEPH_KEY, 
   NTP_KEY, 
   CHAN_KEY, 
   EVETP_KEY, 
   EST_KEY, 
   CASE_KEY, 
   CGP_ID, 
   EVE_DATE, 
   EVE_DATETIME, 
   EVE_RECORDED_DATE, 
   PROCE_TARGET, 
   ACC_KEY, 
   ACCTP_KEY, 
   CARD_KEY, 
   SRV_KEY, 
   CASE_MASTER_KEY, 
   DOC_KEY, 
   DOCL_KEY, 
   STR_KEY, 
   EVE_STATUS_DESC
 from ADS_ETL_OWNER.XC_SMA_PROCESS_EVENTS_MAIN xc
 where (EVE_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
         and EVE_DATE in (to_date('20250910','YYYYMMDD'), to_date('20250910','YYYYMMDD') -1, to_date('20250910','YYYYMMDD')+1)
         and EVE_SOURCE_SYSTEM_ID = 'SMA'
         and EVETP_KEY in (
            Select EVETP_KEY from ADS_OWNER.EVENT_TYPES
         where
            EVENT_TYPES.EVETP_VALID_TO = date '3000-01-01' and
            EVENT_TYPES.EVETP_SOURCE_SYSTEM_ID = 'SMA' and
            EVENT_TYPES.EVETP_SOURCE_SYS_ORIGIN = 'SMA_SYSTEM_EVENTS'
         ))) src
on 
     (trg.EVE_SOURCE_ID = src.EVE_SOURCE_ID
 and trg.EVE_SOURCE_SYSTEM_ID = src.EVE_SOURCE_SYSTEM_ID
 and trg.EVE_SOURCE_SYS_ORIGIN = src.EVE_SOURCE_SYS_ORIGIN
 and trg.EVETP_KEY = src.EVETP_KEY
 and trg.EVE_DATE = src.EVE_DATE)
when matched then update set 
     trg.PT_KEY = src.PT_KEY, 
     trg.PTORI_KEY = src.PTORI_KEY, 
     trg.PT_UNIFIED_KEY = src.PT_UNIFIED_KEY, 
     trg.PT_EMPLOYEE_KEY = src.PT_EMPLOYEE_KEY, 
     trg.PTORI_EMPLOYEE_KEY = src.PTORI_EMPLOYEE_KEY, 
     trg.ID_FINANCIAL_PRODUCT = src.ID_FINANCIAL_PRODUCT, 
     trg.CASEPH_KEY = src.CASEPH_KEY, 
     trg.NTP_KEY = src.NTP_KEY, 
     trg.CHAN_KEY = src.CHAN_KEY, 
     trg.EST_KEY = src.EST_KEY, 
     trg.CASE_KEY = src.CASE_KEY, 
     trg.CGP_ID = src.CGP_ID, 
     trg.EVE_DATETIME = src.EVE_DATETIME, 
     trg.EVE_RECORDED_DATE = src.EVE_RECORDED_DATE, 
     trg.PROCE_TARGET = src.PROCE_TARGET, 
     trg.PROCE_UPDATED_DATETIME = sysdate, 
     trg.PROCE_UPDATE_PROCESS_KEY = 13161748, 
     trg.ACC_KEY = src.ACC_KEY, 
     trg.ACCTP_KEY = src.ACCTP_KEY, 
     trg.CARD_KEY = src.CARD_KEY, 
     trg.SRV_KEY = src.SRV_KEY, 
     trg.CASE_MASTER_KEY = src.CASE_MASTER_KEY, 
     trg.DOC_KEY = src.DOC_KEY, 
     trg.DOCL_KEY = src.DOCL_KEY, 
     trg.STR_KEY = src.STR_KEY, 
     trg.EVE_STATUS_DESC = src.EVE_STATUS_DESC
 where (
     decode( src.PT_KEY,trg.PT_KEY,1,0 ) = 0  or
     decode( src.PTORI_KEY,trg.PTORI_KEY,1,0 ) = 0  or
     decode( src.PT_UNIFIED_KEY,trg.PT_UNIFIED_KEY,1,0 ) = 0  or
     decode( src.PT_EMPLOYEE_KEY,trg.PT_EMPLOYEE_KEY,1,0 ) = 0  or
     decode( src.PTORI_EMPLOYEE_KEY,trg.PTORI_EMPLOYEE_KEY,1,0 ) = 0  or
     decode( src.ID_FINANCIAL_PRODUCT,trg.ID_FINANCIAL_PRODUCT,1,0 ) = 0  or
     decode( src.CASEPH_KEY,trg.CASEPH_KEY,1,0 ) = 0  or
     decode( src.NTP_KEY,trg.NTP_KEY,1,0 ) = 0  or
     decode( src.CHAN_KEY,trg.CHAN_KEY,1,0 ) = 0  or
     decode( src.EST_KEY,trg.EST_KEY,1,0 ) = 0  or
     decode( src.CASE_KEY,trg.CASE_KEY,1,0 ) = 0  or
     decode( src.CGP_ID,trg.CGP_ID,1,0 ) = 0  or
     decode( src.EVE_DATETIME,trg.EVE_DATETIME,1,0 ) = 0  or
     decode( src.EVE_RECORDED_DATE,trg.EVE_RECORDED_DATE,1,0 ) = 0  or
     decode( src.PROCE_TARGET,trg.PROCE_TARGET,1,0 ) = 0  or
     decode( src.ACC_KEY,trg.ACC_KEY,1,0 ) = 0  or
     decode( src.ACCTP_KEY,trg.ACCTP_KEY,1,0 ) = 0  or
     decode( src.CARD_KEY,trg.CARD_KEY,1,0 ) = 0  or
     decode( src.SRV_KEY,trg.SRV_KEY,1,0 ) = 0  or
     decode( src.CASE_MASTER_KEY,trg.CASE_MASTER_KEY,1,0 ) = 0  or
     decode( src.DOC_KEY,trg.DOC_KEY,1,0 ) = 0  or
     decode( src.DOCL_KEY,trg.DOCL_KEY,1,0 ) = 0  or
     decode( src.STR_KEY,trg.STR_KEY,1,0 ) = 0  or
     decode( src.EVE_STATUS_DESC,trg.EVE_STATUS_DESC,1,0 ) = 0) 
when not matched then  insert 
  (trg.EVE_KEY, 
   trg.EVE_SOURCE_ID, 
   trg.EVE_SOURCE_SYSTEM_ID, 
   trg.EVE_SOURCE_SYS_ORIGIN, 
   trg.PT_KEY, 
   trg.PTORI_KEY, 
   trg.PT_UNIFIED_KEY, 
   trg.PT_EMPLOYEE_KEY, 
   trg.PTORI_EMPLOYEE_KEY, 
   trg.ID_FINANCIAL_PRODUCT, 
   trg.CASEPH_KEY, 
   trg.NTP_KEY, 
   trg.CHAN_KEY, 
   trg.EVETP_KEY, 
   trg.EST_KEY, 
   trg.CASE_KEY, 
   trg.CGP_ID, 
   trg.EVE_DATE, 
   trg.EVE_DATETIME, 
   trg.EVE_RECORDED_DATE, 
   trg.PROCE_TARGET, 
   trg.PROCE_INSERTED_DATETIME, 
   trg.PROCE_INSERT_PROCESS_KEY, 
   trg.PROCE_UPDATED_DATETIME, 
   trg.PROCE_UPDATE_PROCESS_KEY, 
   trg.ACC_KEY, 
   trg.ACCTP_KEY, 
   trg.CARD_KEY, 
   trg.SRV_KEY, 
   trg.CASE_MASTER_KEY, 
   trg.DOC_KEY, 
   trg.DOCL_KEY, 
   trg.STR_KEY, 
   trg.EVE_STATUS_DESC
 ) 
      values 
  (ADS_OWNER.PROCESS_EVENTS_S.nextval, 
   src.EVE_SOURCE_ID, 
   src.EVE_SOURCE_SYSTEM_ID, 
   src.EVE_SOURCE_SYS_ORIGIN, 
   src.PT_KEY, 
   src.PTORI_KEY, 
   src.PT_UNIFIED_KEY, 
   src.PT_EMPLOYEE_KEY, 
   src.PTORI_EMPLOYEE_KEY, 
   src.ID_FINANCIAL_PRODUCT, 
   src.CASEPH_KEY, 
   src.NTP_KEY, 
   src.CHAN_KEY, 
   src.EVETP_KEY, 
   src.EST_KEY, 
   src.CASE_KEY, 
   src.CGP_ID, 
   src.EVE_DATE, 
   src.EVE_DATETIME, 
   src.EVE_RECORDED_DATE, 
   src.PROCE_TARGET, 
   sysdate, 
   13161748, 
   sysdate, 
   13161748, 
   src.ACC_KEY, 
   src.ACCTP_KEY, 
   src.CARD_KEY, 
   src.SRV_KEY, 
   src.CASE_MASTER_KEY, 
   src.DOC_KEY, 
   src.DOCL_KEY, 
   src.STR_KEY, 
   src.EVE_STATUS_DESC) 
